In [7]:
import pygame
from pygame.locals import *
import random
import time

pygame.init()
FPS = 120
FramePerSec = pygame.time.Clock()

SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
SPEED = 5
SCORE = 0
COINS_COLLECTED = 0
PREV_MILESTONE = 0 

coin_sound = pygame.mixer.Sound("PYTHON_8 lab_звук монеты.mp3")

DISPLAY = pygame.display.set_mode((400, 600))
DISPLAY.fill((255, 255, 255))
pygame.display.set_caption("GAME")

font = pygame.font.SysFont("Verdana", 60)
font_small = pygame.font.SysFont("Times New Roman", 20)
game_over = font.render("Game Over", True, (0, 0, 0))

background = pygame.image.load("AnimatedStreet.png")
pygame.mixer.music.load("PYTHON_8 lab_background.wav")
pygame.mixer.music.play(-1)

class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("Enemy.png")
        self.image = pygame.transform.scale(self.image, (50, 100))
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(20, SCREEN_WIDTH - 20), 0)

    def move(self):
        global SCORE
        self.rect.move_ip(0, SPEED)
        if self.rect.bottom > 600:
            SCORE += 1
            self.rect.top = 0
            self.rect.center = (random.randint(20, SCREEN_WIDTH - 20), 0) #экранның размерінен асып кетсе жылдамдығы артады қайтып жоғарыдан бастап шығады.

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("Player.png")
        self.image = pygame.transform.scale(self.image, (50, 100))
        self.rect = self.image.get_rect()
        self.rect.center = (207, 520)

    def move(self):
        pressed_keys = pygame.key.get_pressed()
        if self.rect.left > 0 and pressed_keys[K_LEFT]:
            self.rect.move_ip(-5, 0)
        if self.rect.right < SCREEN_WIDTH and pressed_keys[K_RIGHT]:
            self.rect.move_ip(5, 0) #солға және оңға қозғалта аламыз экран шетінен шығып кете алмайды

class Coin(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.respawn()

    def respawn(self):
        weights = [1, 2, 3]  
        self.weight = random.choice(weights) 
       
        image_path = f"coin{self.weight}.jpg"
        self.original_image = pygame.image.load("coin1.png")
        size = 30 + self.weight * 5 
        self.image = pygame.transform.scale(self.original_image, (size, size))
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(20, SCREEN_WIDTH - 20), 0) #әрбір монетаның салмағы болады 

    def move(self):
        self.rect.move_ip(0, SPEED // 2)
        if self.rect.top > SCREEN_HEIGHT:
            self.respawn()

P1 = Player()
E1 = Enemy()
C1 = Coin()

enemies = pygame.sprite.Group()
enemies.add(E1)

coins = pygame.sprite.Group()
coins.add(C1)

all_sprites = pygame.sprite.Group()
all_sprites.add(P1, E1, C1) #барлығын бір топқа жинаймыз

INC_SPEED = pygame.USEREVENT + 1
pygame.time.set_timer(INC_SPEED, 1000) #әр секунд сайын жылдамдықты арттыру

while True:
    for event in pygame.event.get():
        if event.type == INC_SPEED:
            SPEED += 0.3
        if event.type == QUIT:
            pygame.quit() #әрбір секунд сайын 0,3 ке арттырамыз

    DISPLAY.blit(background, (0, 0)) #фон суретінің кординаталары
 
    scores = font_small.render(str(SCORE), True, (0, 0, 0))
    DISPLAY.blit(scores, (10, 10))
    coins_collected_text = font_small.render(f"Coins: {COINS_COLLECTED}", True, (0, 0, 0))
    DISPLAY.blit(coins_collected_text, (300, 10)) #экранда жиналған ұпайлар мен монета сандарын шығарады
    
    for entity in all_sprites:
        entity.move()
        DISPLAY.blit(entity.image, entity.rect) #барлық кластарды іске қосады
        
    if pygame.sprite.spritecollideany(P1, enemies):
        pygame.mixer.music.stop()
        pygame.mixer.Sound("PYTHON_8 lab_crash.wav").play()
        time.sleep(0.7)
        DISPLAY.fill((255, 0, 0))
        DISPLAY.blit(game_over, (30, 250))
        pygame.display.update()
        for entity in all_sprites:
            entity.kill()
        time.sleep(1)
        pygame.quit() #машина мен жау соғысуы

    if pygame.sprite.spritecollideany(P1, coins):
        COINS_COLLECTED += C1.weight
        coin_sound.play()
        C1.respawn() #машинамен монета жинау
     
        if COINS_COLLECTED // 10 > PREV_MILESTONE:
            SPEED += 0.5
            PREV_MILESTONE = COINS_COLLECTED // 10 #егер жинаған монеталарымыз 10нан асса жау 0,5ке көбейеді

    pygame.display.update()
    FramePerSec.tick(FPS)

error: video system not initialized

In [9]:
import pygame
import random
import sys
import time

pygame.init()

WIDTH, HEIGHT = 600, 600
CELL_SIZE = 20
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
FPS = 10

FOOD_COLORS = {
    1: (255, 0, 0),     
    2: (255, 165, 0),   
    3: (255, 255, 0)   
}

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Змейка")
font = pygame.font.SysFont("Times New Roman", 20)
clock = pygame.time.Clock()

snake = [(100, 100), (80, 100), (60, 100)] #бастапқы позициясы
snake_dir = (20, 0) #жыланның қозғалу бағыты

food_pos = None 
food_weight = 1
food_spawn_time = 0 #тамақтың басапқы позициясы салмағы қайта шығу уақыты

score = 0 
level = 1
speed = 10 # ойынышының жинаған ұпайы ойын деңгейі көбейген сайын жылдамдық артады жыланның жылдамдығы

def generate_food():
    while True:
        x = random.randrange(0, WIDTH, CELL_SIZE)
        y = random.randrange(0, HEIGHT, CELL_SIZE)
        if (x, y) not in snake:
            return (x, y), random.choice([1, 2, 3])  #функция рандом жаңа тамақтың позициясын и салмағын шығарады и оның жыланның үстіне шығып кетпеуін бақылайды

food_pos, food_weight = generate_food()
food_spawn_time = time.time()

while True:
   
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit() #ойынның жабылуы
    
    keys = pygame.key.get_pressed()
    if keys[pygame.K_UP] and snake_dir != (0, 20):
        snake_dir = (0, -20)
    if keys[pygame.K_DOWN] and snake_dir != (0, -20):
        snake_dir = (0, 20)
    if keys[pygame.K_LEFT] and snake_dir != (20, 0):
        snake_dir = (-20, 0)
    if keys[pygame.K_RIGHT] and snake_dir != (-20, 0):
        snake_dir = (20, 0) #оңға жүріп баратқанда солға бұрыла алмайды
   
    new_head = (snake[0][0] + snake_dir[0], snake[0][1] + snake_dir[1])
   
    if (
        new_head[0] < 0 or new_head[0] >= WIDTH or
        new_head[1] < 0 or new_head[1] >= HEIGHT or
        new_head in snake
    ):
        break #экраннан шығып кетсе немесе өзі мен қақтығысып қалса, ойын тоқтайды.
   
    snake.insert(0, new_head)

    if new_head == food_pos:
        score += food_weight 
        food_pos, food_weight = generate_food()  
        food_spawn_time = time.time() 
        if score // 4 + 1 > level:
            level += 1
            speed += 2
    else:
        snake.pop() # тамаққа тисе ұпай қосылады және жылдамдық артады и жаңа тамақ пайда болады

    if time.time() - food_spawn_time >= 5:
        food_pos, food_weight = generate_food()
        food_spawn_time = time.time()

    screen.fill(WHITE)

    for segment in snake:
        pygame.draw.rect(screen, GREEN, (segment[0], segment[1], CELL_SIZE, CELL_SIZE))

    food_color = FOOD_COLORS[food_weight]
    pygame.draw.rect(screen, food_color, (food_pos[0], food_pos[1], CELL_SIZE, CELL_SIZE))

    score_text = font.render(f"Счет: {score}", True, BLACK)
    level_text = font.render(f"Уровень: {level}", True, BLACK)
    screen.blit(score_text, (10, 10))
    screen.blit(level_text, (10, 40)) #экранда ұпайлар уровень сандары шығып тұрады
 
    pygame.display.flip()
    clock.tick(speed) #жылдамдықты қарап отырады

In [9]:
import pygame
import sys

pygame.init()

screen = pygame.display.set_mode((800, 600))
pygame.display.set_caption("paint")
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
current_color = BLACK

screen.fill(WHITE)

brush_size = 5
drawing = False
last_pos = None # мышканың жүрісін анықтайды
mode = "brush" 
start_pos = None

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
       
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:
                current_color = RED
            elif event.key == pygame.K_g:
                current_color = GREEN
            elif event.key == pygame.K_b:
                current_color = BLUE
            elif event.key == pygame.K_e:
                mode = "eraser"
            elif event.key == pygame.K_t:
                mode = "brush"
            elif event.key == pygame.K_c:
                mode = "circle"
            elif event.key == pygame.K_q:
                mode = "square"
            elif event.key == pygame.K_p:
                mode = "right_triangle"
            elif event.key == pygame.K_e:
                mode = "equilateral_triangle"
            elif event.key == pygame.K_d:
                mode = "rhombus"
            elif event.key == pygame.K_l:
                mode = "rect"
       
        if event.type == pygame.MOUSEBUTTONDOWN:
            drawing = True
            start_pos = event.pos
            last_pos = event.pos #мышка басылса сурет салынып басталады
        
        if event.type == pygame.MOUSEBUTTONUP:
            end_pos = event.pos #мышка алынса өзгерізссіз қалады

            if mode == "rect": #төртбұрыш
                rect = pygame.Rect(min(start_pos[0], end_pos[0]), min(start_pos[1], end_pos[1]),
                                   abs(start_pos[0] - end_pos[0]), abs(start_pos[1] - end_pos[1]))
                pygame.draw.rect(screen, current_color, rect, width=2)

            elif mode == "circle": #круг
                radius = int(((start_pos[0] - end_pos[0])**2 + (start_pos[1] - end_pos[1])**2) ** 0.5)
                pygame.draw.circle(screen, current_color, start_pos, radius, width=2)

            elif mode == "square": #квадрат
                side = max(abs(end_pos[0] - start_pos[0]), abs(end_pos[1] - start_pos[1]))
                rect = pygame.Rect(start_pos[0], start_pos[1], side, side)
                pygame.draw.rect(screen, current_color, rect, width=2)

            elif mode == "right_triangle": #дұрыс үшбұрыш
                
                x1, y1 = start_pos
                x2, y2 = end_pos
                points = [start_pos, (x1, y2), (x2, y2)]
                pygame.draw.polygon(screen, current_color, points, width=2)

            elif mode == "equilateral_triangle": #тікбұрышты үшбұрыш
                
                x1, y1 = start_pos
                x2, y2 = end_pos
                side = max(abs(x2 - x1), abs(y2 - y1))
                point1 = (x1, y1)
                point2 = (x1 + side, y1)
                point3 = (x1 + side / 2, y1 - (3**0.5/2)*side)
                pygame.draw.polygon(screen, current_color, [point1, point2, point3], width=2)

            elif mode == "rhombus": #ромб салады
                
                x1, y1 = start_pos
                x2, y2 = end_pos
                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2
                dx = abs(x2 - x1) // 2
                dy = abs(y2 - y1) // 2
                points = [
                    (center_x, center_y - dy),
                    (center_x + dx, center_y),
                    (center_x, center_y + dy),
                    (center_x - dx, center_y)
                ]
                pygame.draw.polygon(screen, current_color, points, width=2) #көпбұрыш салады

            drawing = False

        if event.type == pygame.MOUSEMOTION and drawing: #мышка қозғалған кезде жұмыс жасайды сызады өшіреді
            if mode == "brush":
                pygame.draw.line(screen, current_color, last_pos, event.pos, brush_size)
                last_pos = event.pos
            elif mode == "eraser":
                pygame.draw.line(screen, WHITE, last_pos, event.pos, brush_size * 2)
                last_pos = event.pos

    pygame.display.update()

SystemExit: 

C:\Users\tules\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
